# Probabilistic programming with PyMC

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pymc as mc
%matplotlib inline
import matplotlib as mpl
import scipy.stats
from pydataset import data

mpl.rcParams['figure.figsize'] = (6.0, 3.0)
mpl.rcParams['figure.dpi'] = 300

## Utility function to plot the graph of a PyMC model
def show_dag(model):
    dag = mc.graph.dag(model)
    dag.write("graph.png",format="png")
    from IPython.display import Image
    i = Image(filename='graph.png')
    return i


# Neat things

* Demo of MCMC algorithms and their sampling properties: https://chi-feng.github.io/mcmc-demo/
* Hamiltoninan MCMC visually explained (great animations): http://arogozhnikov.github.io/2016/12/19/markov_chain_monte_carlo.html
* [A Conceptual Introduction to Hamiltonian Monte Carlo](https://arxiv.org/abs/1701.02434) An excellent paper on the theory of Hamiltonian Monte Carlo sampling
* [Introduction to MCMC](http://www.inference.org.uk/mackay/erice.pdf) by David Mackay


        
        

# Topic purpose
We will cover probabilistic **inference**. Rather than learning a single set of parameters by optimisation, we can model probability distributions over possible models that might be compatible with our data.  We'll use Monte Carlo sampling to make it simple and easy (if not very efficient) to work with probabilistic models. 


MCMC models:

* **Data**, which we observe as a collection of examples.
* A **model** which has **structure** (a DAG) and **parameters**
* Part of the model is a likelihood function which has "contact" with data; these we will call **observed random variables**
* Part of the model specifies distributions over parameters of the **observed variables**. These are **unobserved random variables**


## PyMC
<a id="pymc"> </a>
We'll use the excellent PyMC module to do the inference. If you have questions about this module, you can read [this tutorial](http://arxiv.org/abs/1507.08050) or the [API docs](https://pymc-devs.github.io/pymc/). 

# Fitting a normal distribution
## Bayesian Normal fitting
We use Monte Carlo sampling to estimate the mean and standard deviation of some data. We generate some synthetic data from a known normal distribution 
$$x \sim \mathcal{N}(-1, 1.5)$$


In [ ]:
## generate data with a known distribution
## this will be our "observed" data
x_data = np.random.normal(-1,1.5, (3000,))

plt.hist(x_data, bins=np.linspace(-5,5,15))
plt.title("Histogram of data")
plt.xlabel("x")
plt.ylabel("Count")

We then create a model in PyMC. We have a single output variable `x`, which is **stochastic** and **observed**, and the data we have observed is `x_data`. As it is observed, we will use the likelihood of the data under different model settings to accept/reject samples in the process.

We have a model:

$$
\mu \sim \mathcal{N}(0, 10^2)\\
\tau \sim \mathcal{\Gamma}(2.0, 20.0)\\
x\sim\mathcal{N}(\mu, \frac{1}{\tau})\\
$$

In [ ]:
# latent variables
mu = mc.Normal('mu', mu=0, tau=1.0/(10*10))  # wide range for prior on means
prec = mc.Gamma('prec', alpha=2.0, beta=20)         # wide range for prior on precision

In [ ]:
import scipy.stats

xs = np.linspace(0, 100, 100)
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
# alpha = 1.0, beta = 20.0
ax.plot(xs, scipy.stats.gamma(a=2.0, scale=20).pdf(xs))
ax.set_xlabel("$\\tau$")
ax.set_ylabel("$p(\\tau)$")
ax.set_title("Precision $\\tau$ prior")


In [ ]:
# observed variable
observed_stochastic = mc.Normal('observed', 
                                 mu=mu, tau=prec, 
                                 observed=True, 
                                 value=x_data)

# if we want to sample from unconditioned prior
#observed_stochastic = mc.Normal('observed',
#mu=mu, tau=prec)

We have to set parents for every node. In this case, we have two parameters, $\mu$ and $\tau = \frac{1}{\sigma^2}$ to specify ($\tau$ is used to make it easier to parameterise priors over normals). We want to infer those, so we also make those stochastic variables, but unobserved (hidden). We specify the type of the distribution (here, `Normal` and `Uniform`) and we must then specify *those* parents. In this case, these are just concrete numbers (but we could go further if we wanted).

In [ ]:
# generate samples with same distribution
# here, we draw 20 samples in each sample
pred_posterior = mc.Normal('predictive', 
                           mu=mu, 
                           tau=prec, 
                           size=20)




We also add a "false" variable that will be used to make draws from the predictive posterior. It is a variable with the same parents as the observed posterior, but unobserved. Here we generate 20 posterior predictive samples for every accepted MCMC sample.

In [ ]:
# display the graphical model
model = mc.Model([mu, prec, observed_stochastic, 
                  pred_posterior])
show_dag(model)

We compile the model and show the graph. We can now draw samples from it, discarding the first portion:

In [ ]:
# sample from the distribution
mcmc = mc.MCMC(model)
mcmc.sample(iter=50000, burn=5000)

The **trace** is the collection of posterior samples, as a straightforward array. We can plot these using the built in visualisation tool:


In [ ]:
# standard trace plot
mc.Matplot.plot(mcmc)

We can also access them directly as arrays and plot them more flexibly (including showing draws from the predictive posterior):

In [ ]:
def trace_hist(trace, name):
        n, bins, patches = plt.hist(trace, normed=True, bins=50)                
        max_n = np.max(n)
        plt.title("Estimate of {var_name}".format(var_name=name))
        
        # draw simple statistics
        ctr_max = 0.5 * (bins[np.argmax(n)] + bins[np.argmax(n)+1])
        plt.axvline(ctr_max, ls='-', color='r', lw=2, label='MAP')
        plt.axvline(np.median(trace), ls='-', color='C1', lw=2, label='Median')
        plt.axvline(np.mean(trace), ls=':', color='k', label='Expected')
        # 90% credible interval
        plt.axvline(np.percentile(trace, 5.0), ls=':', color='C1')
        plt.axvline(np.percentile(trace, 95.0), ls=':', color='C1')
        plt.fill_between(x=[np.percentile(trace, 5.0), np.percentile(trace, 95.0)], y1=max_n,
                          color='C1', alpha=0.2, label='90% credible')
        plt.text(np.mean(trace), 0.5*max_n, 'Mean')
        plt.legend()
        plt.gca().set_frame_on(False)
        
def show_trace(mcmc, vars_):
    ## plot histograms of possible parameter values
    # from the trace
    for var,name in vars_.items():
        plt.figure()
        trace = mcmc.trace(var)[:].ravel()
        trace_hist(trace, name)
        
        
    
    
    

In [ ]:

show_trace(mcmc, {"mu":"mean", 
                  "prec":"precision",
                  "predictive":"predictive posterior"})

## Changes to try:

* Show prior predictive (remove `observed=`)
* Show fixing \mu and \tau to known values
* Adjust $n$ to show effect of prior/posterior
* Show fitting a uniform distribution instead (below)


In [ ]:
# latent variables
x_data = np.random.uniform(-2, 3, size=800)

# Note: I *force* a good starting point (i.e. non-zero probability)
# otherwise PyMC will not be able to form the model
# this is why value=0 and value=100 are used

ctr = mc.Normal('ctr', mu=0, tau=1e-2, 
                value=0)  # wide range for prior on means
width = mc.Gamma('width', alpha=2.0, 
                 beta=200.0, value=100)         # wide range for prior on precision



# uniform variable, in a unknown range
# note use of Lambda to transform parameters
lower = mc.Lambda('upper', lambda ctr=ctr, 
                  width=width: ctr-width)
upper = mc.Lambda('lower', lambda ctr=ctr, 
                  width=width: ctr+width)

observed_stochastic = mc.Uniform('observed', 
                                 upper=upper, 
                                 lower=lower, 
                                 observed=True, 
                                 value=x_data)

pred_posterior = mc.Uniform('predictive', 
                            upper=upper, 
                            lower=lower)

# display the graphical model
model = mc.Model([ctr, width, 
                  observed_stochastic, 
                  pred_posterior, upper, lower])
show_dag(model)

In [ ]:
mcmc = mc.MCMC(model)
for i in range(10):
    mcmc.sample(iter=50000, burn=5000)

In [ ]:
mc.gelman_rubin(mcmc)

In [ ]:
show_trace(mcmc, {"ctr":"centre", "width":"width", "predictive":"Predictive posterior"})

In [ ]:
mc.Matplot.plot(mcmc)

---
# Linear regression

## Graphical models
<a id="graphical"> </a>

Transformations of expressions to graphs is familiar to most computer scientists -- it is an essential part of most optimising compilers. For example, the equation of a straight line might be written as a graph (this is how a compiler would break down the expression):

<img src="imgs/ymxc.png" width="300px">

## Adding unknowns
If we have multiple dependent random variables whose distribution we want to infer, we can draw a graph of dependencies to form a *graphical model*.  This explictly models dependencies between **random variables** (i.e. ones we don't know the value of precisely) and inference can be performed on the entire graph. 

**In CS terms, we are writing expressions down without fixing the variables, and then allowing the distribution of the values to be inferred when we observe data.** This inference process narrows down the likely range a random variable could take on (hopefully!).

In a **probabilistic graphical model**, some nodes in the graph are **observed** -- that is we know their state because we have explicity measured it, and others are **unobserved** -- we know (or have guessed) the form of their distribution but not the parameters of that distribution. Some dependencies are deterministic (i.e. fully defined by the values of their parents), while others are stochastic. We can infer the **posterior** distribution of unobserved nodes by integrating over the possible values that could have occured given the observed values.

We can modify our straight line equation to write a model for **linear regression**:

<img src="imgs/ymxc_stochastic.png">

All we need to do is specify that we expected the output $y$ to be normally distributed around the equation of a line given by $m$ and $c$; we can now **infer** $\sigma, m, c$ from observed data. Or we can fix any of them, and infer the remainder (if, e.g. we knew in advance that $c=0$). Our assumption here is that we will observe data which has a **latent structure** modelled by a linear dependence on a variable $x$, plus some normally-distributed observation noise.

**Note that we must put *some* prior distribution on every stochastic node and we can only observe stochastic nodes.**

----


Let's implement the linear regression model in the intro in practice, using PyMC to build a graphical model and then run MCMC to sample from the posterior (i.e. estimate the distribution of random variables after seeing some evidence).

In [ ]:
### Bayesian Linear Regression with pymc
### We use Monte Carlo sampling to estimate the distribution of a linear function with a normally
### distributed error, given some observed data.
### Vaguely based on: http://matpalm.com/blog/2012/12/27/dead_simple_pymc/ and http://sabermetricinsights.blogspot.co.uk/2014/05/bayesian-linear-regression-with-pymc.html


## generate data with a known distribution
## this will be our "observed" data
x = np.sort(np.random.uniform(0,20, (50,)))
m = 2
c = 15

# Add on some measurement noise, with std. dev. 3.0
epsilon = data = np.random.normal(0, 3, x.shape)
y = m * x + c + epsilon

plt.plot(x,y, '.', label="Datapoints")
plt.plot(x, m*x+c, '--', lw=3, label="True")
plt.legend()
plt.xlabel("x")
plt.xlabel("y")

In [ ]:
## Now, set up the PyMC model

## specify the prior distribution of the unknown line function variables
## Here, we assume a normal distribution over m and c
m_unknown = mc.Normal('m', 0, 0.01)
c_unknown = mc.Normal('c', 0, 0.001)

## specify a prior over the precision (inverse variance) of the error term
# precision = 1/variance
## Here we specify a uniform distribution from 0.001 to 10.0
precision = mc.Uniform('precision', 
                       lower=0.001, 
                       upper=10.0)

# this is just a convenience for plotting
std_dev = mc.Lambda("std_dev", 
                    lambda precision=precision: 
                    np.sqrt(1.0/precision))

# specify the observed input variable
# we use a normal distribution, but this has no effect --
# the values are fixed and the parameters
# never updated; this is just a way of transforming x 
# into a variable pymc can work with
# (it's really a hack)
x_obs = mc.Normal("x_obs", 0, 1 )
                  #value=x, observed=True)

@mc.deterministic(plot=False)
def line(m=m_unknown, c=c_unknown, x=x_obs):
    return x*m+c

# specify the observed output variable (note if use tau instead of sigma, we use the precision paramterisation)
y_obs =  mc.Normal('y_obs', mu=line, 
                   tau=precision, 
                   #value=y, 
                   #observed=True
                  )

model = mc.Model([m_unknown, c_unknown, precision, x_obs, y_obs, std_dev])

# display the graphical model
show_dag(model)

In [ ]:
# sample from the distribution
mcmc = mc.MCMC(model)
mcmc.sample(iter=50000, burn=1000, thin=1)

In [ ]:
show_trace(mcmc, {"m":"m", 
                  "c":"c", 
                  "std_dev":"std_dev"})

## Draws from the posterior predictive model


[<img src="https://imgs.xkcd.com/comics/error_bars.png">](https://xkcd.com/2110)



In [ ]:
## now plot overlaid samples from the linear function
## Note: this *ignores* the error distribution we've estimated
## If we drew samples from the true posterior predictive, we'd see much greater spread
## in possible simulations
ms = mcmc.trace("m")[:]
cs = mcmc.trace("c")[:]

plt.title("Sampled fits")
plt.plot(x, y, '.', label="Observed")
plt.plot(x, x*m+c, '--', label="True")
xf = np.linspace(-20,40,200)
for m,c in zip(ms[::20], cs[::20]):    
    plt.plot(xf, xf*m+c, 'r-', alpha=0.01)
plt.legend()
plt.xlim(-20,40)
plt.ylim(-40,80)

## Simple diagnostics
Is our sampler taking uncorrelated samples? We can look at the **autocorrelation** of the samples. If they are perfectly unbiased, then this should be zero everywhere (no correlation between successive samples)

In [ ]:

mc.Matplot.autocorrelation(mcmc)



# Sampling issues
## Burn-in and thinning

 The **great thing** about MCMC approaches is that you can basically write down your model and then run inference directly. There is no need to derive complex approximations, or to restrict ourselves to limited models for which we can compute answers analytically. Essentially, no maths by hand; everything is done algorithmically.

The **bad thing** about MCMC approaches is that, even though it will do the "right thing" *asymptotically*, the choice of sampling strategy has a very large influence for the kind of sample runs that are practical to execute. Bayesian inference should depend only on the priors and the evidence observed; but MCMC approaches also depend on the sampling strategy used to approximate the posterior. 

### Dealing with biased sampling
MCMC tries to draw **independent, unbiased** samples from the posterior, but the sampling process (like Metropolis), is not inherently unbiased. For example, successive samples in a random walk are correlated and obviously not independent. 

And although the Markov Chain approach (under fairly relaxed assumptions) will asympotically sample from all of the posterior, if the random walk starts off very far from the bulk of the distribution, it will "wander in the wilderness" for some time before reaching significant probability density. This means early samples from the distribution might be unreasonably dense in very low probability regions in the posterior. How "good" the Markov chain is at sampling from the posterior is called **mixing**; some MCMC setups may mix very badly until they get warmed up.

To mitigate these two common issues, there are a couple of standard tricks: 
* **Burn-in**, which ignores the first $n$ samples from an MCMC draw, to make sure the chain is "mixing" well. Typically, several thousand samples might be ignored.
* **Thinnning**, which takes one sample from every $k$ consecutive samples from the chain, to reduce correlation. Values of raound 5-50 are common.

Tuning these is a matter of art!

The code below implements M-H sampling from the lecture notes, and then shows how burn-in and thinning can be applied.

In [ ]:
def metropolis(fx, q, x_init,n):
    # Perform Metropolis MCMC sampling.
    # p(x): a function that can be evaluated anywhere. p(x) returns the value of p at x
    # q(): a function q that draws a sample from a symmetric distribution and returns it
    # x_init: a starting point
    # n: number of samples
    x = x_init
    
    samples = []
    rejected = [] # we only keep the rejected samples to plot them later
    for i in range(n):
        # find a new candidate spot to jump to
        x_prime =  q(x)        
        p_r = fx(x_prime)/fx(x)
        r = np.random.uniform(0,1) 
        # if it's better, go right away
        if r<p_r:            
            x = x_prime
            samples.append(x_prime)
        else:
            rejected.append(x_prime)            
                
    return np.array(samples), np.array(rejected)

In [ ]:
import scipy.stats

# don't worry about how this code works
# test the sampling process
# create an interesting distribution p (just a mixture of two gaussians)
A = np.array([[0.15, 0.9], [-0.3, 2.5]])
p1 = lambda x:scipy.stats.multivariate_normal(mean=[0,0], cov=A).pdf(x)
p2 = lambda x:scipy.stats.multivariate_normal(mean=[3,0], cov=np.eye(2)).pdf(x)
p = lambda x:p1(x)*0.5+p2(x)*0.5
# create a proposal distribution, with std. dev. 0.5
q = lambda x: np.random.normal(x,0.5,(2,))

# make 500 MCMC steps
accept, reject = metropolis(p,q,[10.5, 30], 5000)

In [ ]:
# plot a heatmap of the distribution, along with the
# accepted and rejected samples from that MCMC chain
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1,1,1)
ax.plot(accept[:,0], accept[:,1], label="Path", lw=0.4)
ax.plot(accept[:,0], accept[:,1], 'b.', label='Accepted', markersize=1)
ax.plot(reject[:,0], reject[:,1], 'rx', label='Rejected', markersize=1)
ax.legend()
x,y = np.meshgrid(np.linspace(-5,5,30), np.linspace(-4,4,30))
img = ax.imshow(p(np.dstack([x,y])), extent=[-4,4,-4,4], cmap='viridis', origin='lower')
ax.grid("off")
fig.colorbar(img, ax=ax, fraction=0.046, pad=0.04)
ax.set_title("MCMC sampling with Metropolis-Hastings")        

In [ ]:
## Burn-in and thinning plot

# introduce correlations
y = accept[:,1] 
x = np.arange(len(y))

# discard 400 samples, keep every 8th sample
burn = 400
thin = 8
plt.plot(x[0:burn], y[0:burn], 'r:')
plt.plot(x[burn::thin], y[burn::thin], 'go', markersize=1)
plt.plot(x[burn:], y[burn:], 'k:', alpha=0.1, markersize=0.2)
plt.plot(x[burn:], y[burn:], 'k.', alpha=0.1, markersize=0.2)


plt.axvline(burn, c='r')
plt.text(15,2.5,"Burn-in period")


The red samples are discarded during burn-in, and the green samples (thinned to every 8th sample) are kept during the remainder of the sampling process. This helps to draw unbiased samples from the posterior.

[The PyMC manual](https://pymc-devs.github.io/pymc/modelchecking.html) explains a number of other diagnostic statistics and plots. **None of these are definitive**, but can give skilled MCMC practitioners insight into the operation of the sampling process.
If you're interested in leaning more about MCMC, David Mackay's [book chapter](http://www.inference.phy.cam.ac.uk/mackay/itprnn/ps/356.384.pdf) is a good reference.

----


# Logistic regression example: discrete dependent variable
On ye olde iris dataset, using the four flower measurements to predict whether or not
the species is `setosa` or another type of iris.

We estimate a set of coefficients $\beta_0, \beta_1, \dots$ and use the logistic function to transform the a linear model into a probability for a Bernoulli variable.

In [ ]:

from pydataset import data
from sklearn.model_selection import train_test_split

iris = data("iris")
iris["is_setosa"] = np.where(iris["Species"]=="setosa", 1, 0)


In [ ]:
iris.head()

In [ ]:
# split the data into a train and test set
iris_train, iris_test = train_test_split(iris)
print("Train size", iris_train.shape)
print("Test size", iris_test.shape)

## Model:

We have some coefficients $\beta$, which feed into our logistic function to produce $l$, and $y$ is Bernoulli distributed (0 or 1) with probability $l$.

$$
\beta_i \sim \mathcal{N}(0, 5)\\
l = \frac{1}{1+e^{\beta_0 + \sum_i \beta_i x_i}}\\
y \sim \mathcal{B}(l)\\
$$

In [ ]:
# binary prediction of "is_setosa", using the four attributes
# of the flower configuration

# predictors (standardised)
xs = np.array(iris_train.iloc[:, 0:4])
x_standardised = (xs - xs.mean()) / xs.std()

# observed values
ys = np.array(iris_train["is_setosa"])

# PyMC variable for inputs
x_std = mc.Normal("x_std", 0, 1, 
                  value=x_standardised, 
                  observed=True)

# 4 regression coefficients
betas = mc.Normal("betas", mu=0, tau=1.0/(50*50), 
                  size=5, value=[0,0,0,0,0])

# link function
@mc.deterministic
def logistic(betas=betas, x_std=x_std):
    return 1.0 / (1 + np.exp(-(betas[0] 
                               + np.sum(betas[1:] * x_std))))

# observed output is Bernoulli distributed
y = mc.Bernoulli("y", p=logistic, 
                 observed=True, value=ys)

model = mc.Model([x_std, y,betas, logistic])

In [ ]:
show_dag(model)    

In [ ]:
mcmc = mc.MCMC(model)

## Run the sampler with 5 different chains
mcmc.sample(iter=150000, burn=10000, thin=10)

In [ ]:
fig = plt.figure(figsize=(10,12))
for i in range(5):
    ax = fig.add_subplot(3,2,i+1)    
    trace_hist(mcmc.trace("betas")[:,i], "$\\beta_{i}$".format(i=i))
    
plt.tight_layout()

In [ ]:
import sklearn.metrics

# write link function for use in prediction
def logistic_predict(betas, x_std):
    return 1.0 / (1 + np.exp(-(betas[0] + np.sum(betas[1:] * x_std, axis=1))))

# standardise predictors in test set
test_x = iris_test.iloc[:, 0:4]
test_x = (test_x - np.mean(test_x))/np.std(test_x)

y_true = iris_test["is_setosa"]

## Predictions
We can draw samples from the posterior and then use the regression coefficients to make new predictions. Annoyingly, 
we have to rewrite the logistic function, but this is easy to do.

In [ ]:
# plot for true versus predicted
fig1 = plt.figure()
ax1 = fig1.add_subplot(1,1,1)
ax1.set_xlabel("True")
ax1.set_ylabel("Predicted")
ax1.set_title("True versus predicted")

# plot for ROC curve
fig2 = plt.figure()
ax2 = fig2.add_subplot(1,1,1)
ax2.set_xlabel("FPR")
ax2.set_xlabel("TPR")

confusions = []
beta_trace = mcmc.trace("betas")[:]

# predict 
for i in range(6):
    # choose a random set of betas
    beta_ix = np.random.randint(0, beta_trace.shape[0]-1)
    beta_vec = beta_trace[beta_ix, :]        
    y_pred =  logistic_predict(beta_vec, test_x)    
    ax1.scatter(y_true+np.random.normal(0,0.01,
                                        y_true.shape),
                y_pred,s=0.2)
    # bias is due to unbalanced classes (I think)
    y_class = np.where(y_pred<0.5, 0, 1)
    confusion = sklearn.metrics.confusion_matrix(y_true, y_class)    
    confusions.append(confusion)
        
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(y_true, y_pred)
    ax2.plot(fpr, tpr)
        

## Distribution of confusion matrices
We can show the (samples from) distribution of confusion matrices if we want:

In [ ]:
    
fig = plt.figure()
ax1 = fig.add_subplot(1,2,1)
ax1.imshow(np.mean(confusions, axis=0))
ax2 = fig.add_subplot(1,2,2)
ax2.imshow(np.std(confusions, axis=0))

    

In [ ]:
# show samples from the confusion matrices
confusions = np.array(confusions)
# some tensor reshaping fun...
confusion_pad = np.stack([confusions, np.zeros_like(confusions)]).swapaxes(0,1)
flat = np.concatenate(np.concatenate(confusion_pad, axis=0), axis=1)
plt.imshow(flat, cmap='magma')

----

# Switchpoint model: more complex logic

<img src="poverty_rates.png">

*[Source: https://ourworldindata.org/extreme-history-methods]*

Data not provided, so hand-digitised via https://apps.automeris.io/wpd/



In [ ]:
import pandas as pd
from io import StringIO

# load data from a string

data = StringIO("""year,poverty_rate
1819.8097502972653, 83.88791593695271
1849.6789536266351, 81.646234676007
1869.655172413793, 75.48161120840629
1889.821640903686, 71.6987740805604
1909.6076099881093, 65.67425569176883
1928.8228299643283, 56.42732049036777
1949.7502972651605, 54.8861646234676
1959.6432818073722, 44.09807355516638
1969.7265160523186, 35.69176882661996
1979.8097502972653, 31.62872154115587
1991.6052318668253, 23.782837127845866
2004.922711058264, 13.695271453590195
2001.8787158145064, 17.19789842381782
1999.0249702734839, 19.159369527145344
1995.9809750297266, 19.299474605954472
1987.0392390011891, 24.483362521891436
1989.8929845422117, 24.483362521891436
1983.9952437574316, 27.98598949211906
1980.9512485136743, 33.450087565674266
1992.936979785969, 22.521891418563897""")

poverty_ = pd.read_csv(data)
# deleting the dodgy data point
# uncomment to experiment
# poverty = poverty_.drop(labels=[6])

poverty = poverty_
poverty

In [ ]:
poverty.plot(x='year', y='poverty_rate', kind='scatter')
plt.gca().set_frame_on(False)

### Hypothesis
We model the data with a linear regression, but where there is a switchpoint, where the regression coefficient changes (i.e. piecewise linear with two pieces). We estimate both the regression coefficients at each position and the location of the switchpoint.

$$s \sim \mathcal{N}{(1960, 100)}\\
\beta_0 \sim \mathcal{N}(50, 10)\\
\beta_1 \sim \mathcal{N}(-1, 2)\\
\beta_2 \sim \mathcal{N}(-1, 2)\\
$$

$$
\mu = \begin{cases}
x<s & \beta_0 + \beta_1 (x-s)\\
x>s & \beta_0 + \beta_2 (x-s)\\
\end{cases}
$$


$$
\tau \sim \mathcal{\Gamma}(1, 10) \\
y \sim \mathcal{N}(\mu, \frac{1}{\tau})
$$


In [ ]:

# PyMC variable for inputs
x = mc.Normal("x", mu=0, tau=1, 
              observed=True, value=poverty["year"])

# 3 betas
beta_0 = mc.Normal("beta_0", mu=50, tau=1.0/(10.0))

beta_1 = mc.Normal("beta_1", mu=-1, tau=1.0/(2*2))
beta_2 = mc.Normal("beta_2", mu=-1, tau=1.0/(2*2))

precision = mc.Gamma("precision", alpha=0.5, beta=1)
# alternatively, could postulate uniform
#switch = mc.Uniform("switch", lower=1820, upper=2020)
switch = mc.Normal("switch", mu=1935, 
                   tau=1.0/(50.0*50.0))

# link function
@mc.deterministic
def switch_mu(beta_0=beta_0, beta_1=beta_1, 
              beta_2=beta_2,
              x=x, switch=switch):
    
    return np.where(x<switch, 
                    beta_0 + beta_1 * (x-switch),
                    beta_0 + beta_2 * (x-switch))

std = mc.Lambda("std_dev", 
                lambda precision=precision:np.sqrt(1.0/precision))

y = mc.Normal("y", mu=switch_mu, tau=precision, 
              observed=True, 
              value=poverty["poverty_rate"])

model = mc.Model([x, y, precision, beta_0, 
                  beta_1, beta_2, switch, switch_mu, std])

In [ ]:
show_dag(model)

In [ ]:
mcmc = mc.MCMC(model)
mcmc.sample(iter=200000, burn=2000, thin=20)
            

In [ ]:
# standard trace plot
mc.Matplot.plot(mcmc)

In [ ]:
show_trace(mcmc, {"beta_0":"intercept", 
                  "beta_1":"slope left",
                  "beta_2":"slope right",
                  "switch":"switchpoint (year)",
                 "std_dev":"std. dev."})

In [ ]:
poverty.plot(x='year', y='poverty_rate', kind='scatter')
ax = plt.gca()
ax.set_frame_on(False)
ax.set_ylabel("Global poverty rate")
ax.set_xlim(1800,2020)
ax.set_ylim(0,100)


beta_0_trace = mcmc.trace("beta_0")[:]
beta_1_trace = mcmc.trace("beta_1")[:]
beta_2_trace = mcmc.trace("beta_2")[:]
switch_trace = mcmc.trace("switch")[:]
tau_trace = mcmc.trace("precision")[:]


for i in range(100):
    ix = np.random.randint(0, beta_0_trace.shape[0]-1)
    s = switch_trace[ix]    
    
    tau = tau_trace[ix]
    x1 = np.clip(s-200, 1800, 2020)
    x2 = s
    x3 = np.clip(s+200, 1800, 2020)
    y1 = beta_0_trace[ix] + beta_1_trace[ix] * (x1-s)
    y2 = beta_0_trace[ix] + beta_1_trace[ix] * (x2-s)
    y3 = beta_0_trace[ix] + beta_2_trace[ix] * (x3-s)
    ax.plot([x1,x2,x3], [y1,y2,y3], 'k', lw=0.05)
    
    
    
    


----

# A simple mixture model: discrete + continuous latent variables
## When things get tricky

We can include both **discrete** and **continuous** variables. A very important case is where we have a **mixture model**. That is, we believe our observations come from one of a number of distributions. For example, in modelling human heights, we might expect height to be normally distributed, but to have two different distributions for men and women.

<img src="imgs/mixture.png">

It is very straightforward to add this to a PyMC graphical model; it is just another random variable to infer. However, sampling is another matter.

In [ ]:
## Adapted from the example given at 
## http://stackoverflow.com/questions/18987697/how-to-model-a-mixture-of-3-normals-in-pymc

n = 3
ndata = 100


## Generate synthetic mixture-of-normals data, 
# with means at -50,0,+50, and std. dev of 5,10,1
v = np.random.randint( 0, n, ndata)
data = ((v==0)*(np.random.normal(50,5,ndata)) + 
        (v==1)*(np.random.normal(-50,10,ndata)) + 
        (v==2)*np.random.normal(0,1,ndata))


## Plot the original data
plt.hist(data, bins=50);  

In [ ]:
## A Dirichlet distribution specifies the distribution over categories
## All 1 means that every category is equally likely
dd = mc.Dirichlet('dd', theta=(1,)*n)

## This variable "selects" the category (i.e. the normal distribution)
## to use. The Dirichlet distribution sets the prior over the categories.
category = mc.Categorical('category', 
                          p=dd, size=ndata)

## Now we set our priors the precision and mean of each normal distribution
## Note the use of "size" to generate a **vector** of variables 
# (i.e. one for each category)

## We expect the precision of each normal to be Gamma distributed 
# (this mainly forces it to be positive!)
precs = mc.Gamma('precs', alpha=1, 
                 beta=10, size=n)

## And the means of the normal to be normally distributed, with a precision of 0.001 
# (i.e. std. dev 1000)
means = mc.Normal('means', 0, 1.0/(100*100), size=n)

## These deterministic functions link the means of the observed distribution 
# to the categories
## They just select one of the elements of the mean/precision vector, 
# given the current value of category
## The input variables must be specified in the parameters, so that 
# PyMC knows which variables to pass to it
@mc.deterministic
def mean(category=category, means=means):
    return means[category]

@mc.deterministic
def prec(category=category, precs=precs):
    return precs[category]

In [ ]:
## Now we specify the variable we observe -- which is normally distributed, *but*
## we don't know the mean or precision. 
# Instead, we pass the **functions** mean() and pred()
## which will be used at each sampling step.
## We specify the observed values of this node, and tell PyMC these are observed 
## This is all that is needed to specify the model
obs = mc.Normal('obs', mean, prec, 
                value=data, observed = True)

## Now we just bundle all the variables together for PyMC
model = mc.Model({'dd': dd,
              'category': category,
              'precs': precs,
              'means': means,
              'obs': obs})

In [ ]:

show_dag(model)    


In [ ]:
mcmc = mc.MCMC(model)

## Now we tell the sampler what method to use
## Metropolis works well, but we must tell PyMC to use a specific
## discrete sampler for the category variable to get good results in a reasonable time
mcmc.use_step_method(mc.AdaptiveMetropolis, 
                     model.means)
mcmc.use_step_method(mc.AdaptiveMetropolis,
                     model.precs)
mcmc.use_step_method(mc.DiscreteMetropolis, 
                     model.category) ## this step is key!
mcmc.use_step_method(mc.AdaptiveMetropolis, 
                     model.dd)

## Run the sampler with 5 different chains
mcmc.sample(iter=150000, burn=1000)


In [ ]:
plt.figure()
plt.hist(mcmc.trace('means', chain=None).gettrace()[:], normed=True, bins=np.linspace(-100,100,50))
plt.title("Estimated means")
plt.legend(['Component 1', 'Component 2', 'Component 3'])
plt.figure()
## show the result in terms of std. dev. (i.e sqrt(1.0/precision))
plt.title("Estimated std. dev")
plt.hist(np.sqrt(1.0/mcmc.trace('precs', chain=None).gettrace()[:]), normed=True, 
         bins=np.linspace(0,15,50))
plt.legend(['Component 1', 'Component 2', 'Component 3'])

# Imputation in quadratic regression
<a id="imputation"> </a>

In PyMC, variables can be **observed** (fixed) or **unobserved** (random). PyMC cycles through the array of known values for the **observed** variables and updates the rest of the graph.


PyMC implements this using **imputation**, where certain missing values in an observed variable can be inferred (*imputed*) from the rest of the model. This creates new random variables and then infers the missing values. **Masked arrays** are used to implement imputation; these allow arrays to have "blank" values, that PyMC can fill in automatically.

This approach creates one new random variable per missing data item; this can create very large models if you are not careful!




In [ ]:
## Example, using very simple quadratic regression model 
import numpy.ma as ma # masked array support

## generate the data for the regression
x = np.sort(np.random.uniform(0, 20, (50,)))
m = 2
c = 15
# Add on some measurement noise, with std. dev. 3.0
epsilon = data = np.random.normal(0, 200, x.shape)
y = m * x * x + c + epsilon

In [ ]:
## Now the imputation; we will try and infer missing some missing values of y (we still have the corresponding x)
## mark last three values of y invalid
y_impute = y[:]

n_missing = 6
impute_ixs = np.sort(np.random.randint(0, len(y)-1, size=n_missing))
y_impute[impute_ixs] = 0
y_impute = ma.masked_equal(y_impute,0)
print("Y masked for imputation:", y_impute) # we will see the last three entries with --

In [ ]:
# create the model (exactly as before, except we switch "y_impute" for "y")
m_unknown = mc.Normal('m', 0, 0.01)
c_unknown = mc.Normal('c', 0, 0.001)
precision = mc.Gamma('precision', alpha=1, beta=5)
std = mc.Lambda('std_dev', lambda precision=precision: np.sqrt(1.0/precision))
x_obs = mc.Normal("x_obs", 0, 1, 
                  value=x, observed=True)

@mc.deterministic(plot=False)
def line(m=m_unknown, c=c_unknown, x=x_obs):
    return x*x*m+c

y_obs =  mc.Normal('y_obs', mu=line, 
                   tau=precision, value=y_impute, 
                   observed=True)
model = mc.Model([m_unknown, c_unknown, std, 
                  precision, x_obs, y_obs])

In [ ]:
# sample from the distribution
mcmc = mc.MCMC(model)
mcmc.sample(iter=100000, burn=5000, thin=10)

In [ ]:
## now we will have three entries in the y_obs trace from this run
y_trace = mcmc.trace('y_obs')[:]
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

## the original data
ax.plot(x, y, '.', label="Data")

ax.plot(x, x*x*m+c, ':', label="True")

m_sample = mcmc.trace("m")[:]
c_sample = mcmc.trace("c")[:]

for i in range(20):
    m_post = np.random.choice(m_sample)
    c_post = np.random.choice(c_sample)
    ax.plot(x, x*x*m_post + c_post, "g", alpha=0.1,
           label="Posterior" if i==0 else None)
    


# samples from posterior predicted for the missing values of y
for i in range(len(impute_ixs)):
        
    ax.axvline(x[impute_ixs[i]], c='C1', alpha=0.1, label="Imputed" if i==0 else None)
    # plot the actual imputed data points
    ax.scatter(np.tile(x[impute_ixs[i]], 
                        (len(y_trace), 1)), 
                        y_trace[:,i], s=2, c='C3', marker='_', 
                        alpha=0.25)
              
    # uncomment to add box plots
    #ax.boxplot([y_trace[:,i]], positions = 
    #           [x[impute_ixs[i]]], widths=2, bootstrap=200,
    #          notch=True, showfliers=False )

ax.set_xlim(-1,25)
ax.set_xticks(np.arange(0,25,5))
ax.set_xticklabels(np.arange(0,25,5))
ax.legend()

In [ ]:
show_trace(mcmc, {"std_dev":"Standard deviation", "m":"m", "c":"c"})



------------